# Data Preparation
We have five raw datasets, each of which comes in different formats. The goal of this section is to prepare the datasets into a unified `DataFrame` with features needed by feature extraction and the same dates. Standardization and pipeline preprocessing would be in the next section.

In [33]:
import pandas as pd 
import numpy as np 

## AAPL

In [34]:
aapl = pd.read_csv('../raw_data/AAPL.csv')

# Target the time frame: 2013/05/06 - 2023/04/26
aapl = aapl[(aapl['Date'] >= '2013-05-06') & (aapl['Date'] <= '2023-04-26')]

# Get target features 
aapl_features = aapl[['Date', 'Open', 'Close', 'High', 'Low', 'Volume']]
aapl_features.shape

(2512, 6)

## Macroeconomic Variables

### US Treasury Bill Bond (13 Weeks)

- Typical Price of US Treasury Bill Bond (13 Weeks)

In [35]:
from sklearn.preprocessing import FunctionTransformer
from sklearn.compose import ColumnTransformer

def get_typical_price(X): 
    return np.average(X, axis=1)

risk_free = pd.read_csv('../raw_data/US_Treasury_Bond_13weeks.csv')
risk_free = risk_free.dropna()
risk_free = risk_free[risk_free['Date'] >= '2013-05-06']
transformer = FunctionTransformer(get_typical_price)
typical_price = transformer.fit_transform(risk_free[['High', 'Close', 'Low']])
risk_free = pd.DataFrame({
    'Date': risk_free['Date'], 
    'Typical Price': typical_price
})
risk_free.shape

(2512, 2)

### S&P 500 Index

In [36]:
sp500 = pd.read_csv('../raw_data/sp500_index.csv')
sp500 = sp500[sp500['Date'] <= '2023-04-26']
sp500.shape

(2512, 2)

## Fundamentals

### Valuation measures
- PeRatio: Share price over earnings per share
- PsRatio: Share price over sales per share
- PbRatio: Share price over book per share
- Enterprise value to revenue ratio 
- Enterprise value to EDITDA(Earnings Before Interest, Taxes, Depreciation, and Amortization) ratio

In [37]:
from datetime import datetime

val_measure = pd.read_csv('../raw_data/AAPL_quarterly_valuation_measures.csv')
# print(val_measure.columns)
peratio = val_measure[val_measure.name == 'PeRatio'].drop(columns=['ttm', 'name'])
psratio = val_measure[val_measure.name == 'PsRatio'].drop(columns=['ttm', 'name'])
pbratio = val_measure[val_measure.name == 'PbRatio'].drop(columns=['ttm', 'name'])
enterprise_val_revenue_ratio = val_measure[val_measure.name == 'EnterprisesValueRevenueRatio'].drop(columns=['ttm', 'name'])
enterprise_val_ebitda_ratio = val_measure[val_measure.name == 'EnterprisesValueEBITDARatio'].drop(columns=['ttm', 'name'])
feats = [peratio, psratio, pbratio, enterprise_val_revenue_ratio, enterprise_val_ebitda_ratio]
names = ['PeRatio', 'PsRatio', 'PbRatio', 'EnterpriseValueRevenueRatio', 'EnterpriseValueEDITDARatio']
dates = sp500.Date 

def quarterly_to_daily(quarter_data, dates, name):
    target_date = []
    for d in dates: 
        target_date.append(np.max(quarter_data.columns[quarter_data.columns <= d]))
    daily_data = np.zeros(dates.shape[0])
    for i, d in enumerate(target_date): 
        daily_data[i] = quarter_data[d]
    return pd.DataFrame({
        'Date': dates,
        name: daily_data
    })

def transform_date_format(X): 
    '''
    Transform the date format of input data's columns to format yyyy-mm-dd

    Args: 
        X: pd.DataFrame where the columns should be quarterly dates in the format mm/dd/yyyy
    Returns: 
        X_transformed: pd.DataFrame where the columns would be quarterly dates in the format yyyy-mm-dd
    '''
    dates = []
    for d in X.columns: 
        date_obj = datetime.strptime(d, '%m/%d/%Y')
        dates.append(date_obj.strftime('%Y-%m-%d'))
    X_transformed = pd.DataFrame(X.values, columns=dates)
    return X_transformed

for i, feat in enumerate(feats): 
    feats[i] = quarterly_to_daily(transform_date_format(feat), dates, names[i])

for i, feat in enumerate(feats): 
    print(feats[i].shape)

(2512, 2)
(2512, 2)
(2512, 2)
(2512, 2)
(2512, 2)


### Financials

- BasicEPS: Earning Per Share

In [38]:
dates = sp500.Date 
fin_measure = pd.read_csv('../raw_data/AAPL_quarterly_financials.csv')
# print(fin_measure.columns)
eps = fin_measure[fin_measure.name == 'BasicEPS'].drop(columns=['ttm', 'name'])
eps = quarterly_to_daily(transform_date_format(eps), dates, 'BasicEPS')
print(eps.shape)

(2512, 2)


## Concatenate all basic features

In [55]:
joined_data = pd.merge(aapl_features, sp500, on='Date')
for f in feats: 
    joined_data = pd.merge(joined_data, f, on='Date')
joined_data = pd.merge(joined_data, eps, on='Date')
joined_data.to_csv('../prepared_data/data.csv')